# 基于滑动窗口的数据处理
## 滑动窗口：
假设:  
`Context`为：[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21 ,22]  
`Query`为:[a, b, c, d, e, f, g]  
那么`input`：  
[CLS][a, b, c, d, e, f, g][SEP][1, 2, 3, 4, 5, 6, 7, 8, 9, 10][SEP]    
[CLS][a, b, c, d, e, f, g][SEP][11, 12, 13, 14, 15, 16, 17, 18, 19, 20][SEP]    
[CLS][a, b, c, d, e, f, g][SEP][21, 22, pad, pad, pad, pad, pad, pad, pad, pad][SEP]
## 带overflow的滑动窗口：
上面提到的滑动窗口假如`Answer`在`Context`9到12,那么答案就无法得到，所以引入带overflow的滑动窗口，即：  
[CLS][a, b, c, d, e, f, g][SEP][1, 2, 3, 4, 5, 6, 7, 8, 9, 10][SEP]    
[CLS][a, b, c, d, e, f, g][SEP][9, 10, 11, 12, 13, 14, 15, 16, 17, 18][SEP]  
重叠值overflow = 2, 这是个超参数，假如值过大那么需要的显存会变大，假如值过小那么预测结果会变差。

# 基于滑动窗口的机器阅读理解任务实现

## step1 导入相关包

In [45]:
from transformers import AutoTokenizer, AutoModelForQuestionAnswering, Trainer, TrainingArguments, DefaultDataCollator
from datasets import load_dataset

## step2 加载数据集

In [2]:
dataset = load_dataset('../dataset/cmrc2018')
dataset

DatasetDict({
    train: Dataset({
        features: ['id', 'context', 'question', 'answers'],
        num_rows: 10142
    })
    validation: Dataset({
        features: ['id', 'context', 'question', 'answers'],
        num_rows: 3219
    })
    test: Dataset({
        features: ['id', 'context', 'question', 'answers'],
        num_rows: 1002
    })
})

## step3 数据预处理

In [3]:
tokenizer = AutoTokenizer.from_pretrained('hfl/chinese-macbert-base')
tokenizer

'(MaxRetryError("HTTPSConnectionPool(host='huggingface.co', port=443): Max retries exceeded with url: /hfl/chinese-macbert-base/resolve/main/tokenizer_config.json (Caused by ProxyError('Cannot connect to proxy.', ConnectionResetError(10054, '远程主机强迫关闭了一个现有的连接。', None, 10054, None)))"), '(Request ID: 2d9baa21-9ac6-4a89-9417-030a6821b320)')' thrown while requesting HEAD https://huggingface.co/hfl/chinese-macbert-base/resolve/main/tokenizer_config.json
Retrying in 1s [Retry 1/5].
'(MaxRetryError("HTTPSConnectionPool(host='huggingface.co', port=443): Max retries exceeded with url: /hfl/chinese-macbert-base/resolve/main/tokenizer_config.json (Caused by ProxyError('Cannot connect to proxy.', ConnectionResetError(10054, '远程主机强迫关闭了一个现有的连接。', None, 10054, None)))"), '(Request ID: a71d2d34-0c5a-46e8-ac99-75e4bf192092)')' thrown while requesting HEAD https://huggingface.co/hfl/chinese-macbert-base/resolve/main/tokenizer_config.json
Retrying in 2s [Retry 2/5].
'(MaxRetryError("HTTPSConnectionPool(h

BertTokenizerFast(name_or_path='hfl/chinese-macbert-base', vocab_size=21128, model_max_length=1000000000000000019884624838656, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'}, clean_up_tokenization_spaces=False, added_tokens_decoder={
	0: AddedToken("[PAD]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	100: AddedToken("[UNK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	101: AddedToken("[CLS]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	102: AddedToken("[SEP]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	103: AddedToken("[MASK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
}
)

In [4]:
sample_dataset = dataset['train'].select(range(10))
tokenized_sample_dataset = tokenizer(
    text = list(sample_dataset['question']),
    text_pair = list(sample_dataset['context']),
    return_offsets_mapping = True,    #返回每个token的起始位置
    return_overflowing_tokens = True, #允许数据集的样本被拆分为多个样本
    stride = 128,                     #滑动窗口重叠的长度 
    max_length = 384,
    truncation = 'only_second',
    padding = 'max_length'
)
tokenized_sample_dataset.data.keys()

dict_keys(['input_ids', 'token_type_ids', 'attention_mask', 'offset_mapping', 'overflow_to_sample_mapping'])

In [5]:
print(tokenized_sample_dataset['overflow_to_sample_mapping'],'\n', len(tokenized_sample_dataset['overflow_to_sample_mapping']))

[0, 0, 0, 1, 1, 1, 2, 2, 2, 3, 3, 3, 4, 4, 4, 5, 5, 5, 6, 6, 6, 7, 7, 7, 8, 8, 8, 9, 9] 
 29


In [6]:
for seq in tokenizer.batch_decode(tokenized_sample_dataset['input_ids'][0:3]):  #查看输入序列的滑动窗口情况
    print(seq, '\n')

[CLS] 范 廷 颂 是 什 么 时 候 被 任 为 主 教 的 ？ [SEP] 范 廷 颂 枢 机 （ ， ） ， 圣 名 保 禄 · 若 瑟 （ ） ， 是 越 南 罗 马 天 主 教 枢 机 。 1963 年 被 任 为 主 教 ； 1990 年 被 擢 升 为 天 主 教 河 内 总 教 区 宗 座 署 理 ； 1994 年 被 擢 升 为 总 主 教 ， 同 年 年 底 被 擢 升 为 枢 机 ； 2009 年 2 月 离 世 。 范 廷 颂 于 1919 年 6 月 15 日 在 越 南 宁 平 省 天 主 教 发 艳 教 区 出 生 ； 童 年 时 接 受 良 好 教 育 后 ， 被 一 位 越 南 神 父 带 到 河 内 继 续 其 学 业 。 范 廷 颂 于 1940 年 在 河 内 大 修 道 院 完 成 神 学 学 业 。 范 廷 颂 于 1949 年 6 月 6 日 在 河 内 的 主 教 座 堂 晋 铎 ； 及 后 被 派 到 圣 女 小 德 兰 孤 儿 院 服 务 。 1950 年 代 ， 范 廷 颂 在 河 内 堂 区 创 建 移 民 接 待 中 心 以 收 容 到 河 内 避 战 的 难 民 。 1954 年 ， 法 越 战 争 结 束 ， 越 南 民 主 共 和 国 建 都 河 内 ， 当 时 很 多 天 主 教 神 职 人 员 逃 至 越 南 的 南 方 ， 但 范 廷 颂 仍 然 留 在 河 内 。 翌 年 管 理 圣 若 望 小 修 院 ； 惟 在 1960 年 因 捍 卫 修 院 的 自 由 、 自 治 及 拒 绝 政 府 在 修 院 设 政 治 课 的 要 求 而 被 捕 。 1963 年 4 月 5 日 ， 教 宗 任 命 范 廷 颂 为 天 主 教 北 宁 教 区 主 教 ， 同 年 8 月 15 日 就 任 ； 其 牧 铭 为 「 我 信 [SEP] 

[CLS] 范 廷 颂 是 什 么 时 候 被 任 为 主 教 的 ？ [SEP] 越 南 民 主 共 和 国 建 都 河 内 ， 当 时 很 多 天 主 教 神 职 人 员 逃 至 越 南 的 南 方 ， 但 范 廷 颂 仍 然 留 在 河 内 。 翌 年 管 理 圣 若 望 小 修 院 ； 惟 在 1960 年 因 捍 卫 修 院 的 自 由 、 自 治 及 

In [7]:
#从上一个block可以看到第一个Context被分成了三块，
#这里可以看到对于Context的offset_mapping,第一个从(0,1)开始
#第二个就从上一句的overflow_tokens开始 这里是(266， 267)
for offset_map in tokenized_sample_dataset['offset_mapping'][0:2]:
    print(offset_map, '\n')

[(0, 0), (0, 1), (1, 2), (2, 3), (3, 4), (4, 5), (5, 6), (6, 7), (7, 8), (8, 9), (9, 10), (10, 11), (11, 12), (12, 13), (13, 14), (14, 15), (0, 0), (0, 1), (1, 2), (2, 3), (3, 4), (4, 5), (5, 6), (6, 7), (7, 8), (8, 9), (9, 10), (10, 11), (11, 12), (12, 13), (13, 14), (14, 15), (15, 16), (16, 17), (17, 18), (18, 19), (19, 20), (20, 21), (21, 22), (22, 23), (23, 24), (24, 25), (25, 26), (26, 27), (27, 28), (28, 29), (29, 30), (30, 34), (34, 35), (35, 36), (36, 37), (37, 38), (38, 39), (39, 40), (40, 41), (41, 45), (45, 46), (46, 47), (47, 48), (48, 49), (49, 50), (50, 51), (51, 52), (52, 53), (53, 54), (54, 55), (55, 56), (56, 57), (57, 58), (58, 59), (59, 60), (60, 61), (61, 62), (62, 63), (63, 67), (67, 68), (68, 69), (69, 70), (70, 71), (71, 72), (72, 73), (73, 74), (74, 75), (75, 76), (76, 77), (77, 78), (78, 79), (79, 80), (80, 81), (81, 82), (82, 83), (83, 84), (84, 85), (85, 86), (86, 87), (87, 91), (91, 92), (92, 93), (93, 94), (94, 95), (95, 96), (96, 97), (97, 98), (98, 99), (

In [33]:
print(tokenized_sample_dataset.sequence_ids(0))

[None, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, None, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 

start_context和end_context:通过对句子的id找到对应的offset_mapping的Context的起始和结束位置。这样就能划分开Question和Context了

In [18]:
offset1 = tokenized_sample_dataset['offset_mapping'][0]
offset2 = tokenized_sample_dataset.get('offset_mapping')[0]
print(offset1, '\n', offset2)

[(0, 0), (0, 1), (1, 2), (2, 3), (3, 4), (4, 5), (5, 6), (6, 7), (7, 8), (8, 9), (9, 10), (10, 11), (11, 12), (12, 13), (13, 14), (14, 15), (0, 0), (0, 1), (1, 2), (2, 3), (3, 4), (4, 5), (5, 6), (6, 7), (7, 8), (8, 9), (9, 10), (10, 11), (11, 12), (12, 13), (13, 14), (14, 15), (15, 16), (16, 17), (17, 18), (18, 19), (19, 20), (20, 21), (21, 22), (22, 23), (23, 24), (24, 25), (25, 26), (26, 27), (27, 28), (28, 29), (29, 30), (30, 34), (34, 35), (35, 36), (36, 37), (37, 38), (38, 39), (39, 40), (40, 41), (41, 45), (45, 46), (46, 47), (47, 48), (48, 49), (49, 50), (50, 51), (51, 52), (52, 53), (53, 54), (54, 55), (55, 56), (56, 57), (57, 58), (58, 59), (59, 60), (60, 61), (61, 62), (62, 63), (63, 67), (67, 68), (68, 69), (69, 70), (70, 71), (71, 72), (72, 73), (73, 74), (74, 75), (75, 76), (76, 77), (77, 78), (78, 79), (79, 80), (80, 81), (81, 82), (82, 83), (83, 84), (84, 85), (85, 86), (86, 87), (87, 91), (91, 92), (92, 93), (93, 94), (94, 95), (95, 96), (96, 97), (97, 98), (98, 99), (

In [35]:
def process_func(example):
    tokenized_example = tokenizer(
        text = list(example['question']),
        text_pair = list(example['context']),
        return_offsets_mapping = True,
        return_overflowing_tokens = True,
        truncation = 'only_second',
        padding = 'max_length',
        max_length = 384,
        stride = 128,
    )
    sample_mapping = tokenized_example.pop('overflow_to_sample_mapping') #输出对应的是第几个batch的样本
    start_positions = []
    end_positions = []
    example_ids = []

    for idx, _ in enumerate(sample_mapping):
        answer = example['answers'][sample_mapping[idx]]
        start_char = answer['answer_start'][0]
        end_char = start_char + len(answer['text'][0])
        
        start_context = tokenized_example.sequence_ids(idx).index(1)
        end_context   = tokenized_example.sequence_ids(idx).index(None, start_context) - 1
        
        #判断答案是否在context内
        offset = tokenized_example.get('offset_mapping')[idx]
        if offset[end_context][1] < start_char or offset[start_context][0] > end_char:
            start_token_pos = 0
            end_token_pos = 0
        else:
            start_token_pos = start_context
            end_token_pos   = end_context
            while start_token_pos < end_context and offset[start_token_pos][0] < start_char:
                start_token_pos += 1
            while end_token_pos > start_context and offset[end_token_pos][1] > end_char:  
                end_token_pos -= 1
        start_positions.append(start_token_pos)
        end_positions.append(end_token_pos)
        example_ids.append(example['id'][sample_mapping[idx]])
        tokenized_example['offset_mapping'][idx] = [                        #这里对[idx]进行赋值
            (o if tokenized_example.sequence_ids(idx)[k] == 1 else None)
            for k, o in enumerate(tokenized_example['offset_mapping'][idx])
        ]
    
    tokenized_example["example_ids"] = example_ids
    tokenized_example["start_positions"] = start_positions
    tokenized_example["end_positions"] = end_positions
    return tokenized_example

In [36]:
tokenized_datasets = dataset.map(process_func, batched=True, remove_columns=dataset['train'].column_names)
tokenized_datasets

Map:   0%|          | 0/10142 [00:00<?, ? examples/s]

Map:   0%|          | 0/3219 [00:00<?, ? examples/s]

Map:   0%|          | 0/1002 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'offset_mapping', 'example_ids', 'start_positions', 'end_positions'],
        num_rows: 19189
    })
    validation: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'offset_mapping', 'example_ids', 'start_positions', 'end_positions'],
        num_rows: 6327
    })
    test: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'offset_mapping', 'example_ids', 'start_positions', 'end_positions'],
        num_rows: 1988
    })
})

In [38]:
print(tokenized_datasets['train']['offset_mapping'][0])

[None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, [0, 1], [1, 2], [2, 3], [3, 4], [4, 5], [5, 6], [6, 7], [7, 8], [8, 9], [9, 10], [10, 11], [11, 12], [12, 13], [13, 14], [14, 15], [15, 16], [16, 17], [17, 18], [18, 19], [19, 20], [20, 21], [21, 22], [22, 23], [23, 24], [24, 25], [25, 26], [26, 27], [27, 28], [28, 29], [29, 30], [30, 34], [34, 35], [35, 36], [36, 37], [37, 38], [38, 39], [39, 40], [40, 41], [41, 45], [45, 46], [46, 47], [47, 48], [48, 49], [49, 50], [50, 51], [51, 52], [52, 53], [53, 54], [54, 55], [55, 56], [56, 57], [57, 58], [58, 59], [59, 60], [60, 61], [61, 62], [62, 63], [63, 67], [67, 68], [68, 69], [69, 70], [70, 71], [71, 72], [72, 73], [73, 74], [74, 75], [75, 76], [76, 77], [77, 78], [78, 79], [79, 80], [80, 81], [81, 82], [82, 83], [83, 84], [84, 85], [85, 86], [86, 87], [87, 91], [91, 92], [92, 93], [93, 94], [94, 95], [95, 96], [96, 97], [97, 98], [98, 99], [99, 100], [100, 101], [101, 105], [105, 106],

## step4 获取模型输出

In [ ]:
import numpy as np
import collections

def get_result(start_logits, end_logits, exmaples, features):

    predictions = {}
    references = {}

    # example 和 feature的映射
    example_to_feature = collections.defaultdict(list)
    for idx, example_id in enumerate(features["example_ids"]):
        example_to_feature[example_id].append(idx)

    # 最优答案候选
    n_best = 20
    # 最大答案长度
    max_answer_length = 30

    for example in exmaples:
        example_id = example["id"]
        context = example["context"]
        answers = []
        for feature_idx in example_to_feature[example_id]:
            start_logit = start_logits[feature_idx]
            end_logit = end_logits[feature_idx]
            offset = features[feature_idx]["offset_mapping"]
            start_indexes = np.argsort(start_logit)[::-1][:n_best].tolist()
            end_indexes = np.argsort(end_logit)[::-1][:n_best].tolist()
            for start_index in start_indexes:
                for end_index in end_indexes:
                    if offset[start_index] is None or offset[end_index] is None:  #位置出现在了Question或其它标识符上
                        continue
                    if end_index < start_index or end_index - start_index + 1 > max_answer_length:  # 确保答案长度小于最大长度 且大于0
                        continue
                    answers.append({
                        "text": context[offset[start_index][0]: offset[end_index][1]],  #答案文本
                        "score": start_logit[start_index] + end_logit[end_index]
                    })
        if len(answers) > 0:
            best_answer = max(answers, key=lambda x: x["score"])
            predictions[example_id] = best_answer["text"]
        else:
            predictions[example_id] = ""
        references[example_id] = example["answers"]["text"]

    return predictions, references

## stpe5 评估函数
用nltk库实现  
第一次使用前需要 nltk.download('punkt')

In [ ]:
# import nltk
# nltk.download('punkt')

In [41]:
from cmrc_eval import evaluate_cmrc

def metirc(pred):
    start_logits, end_logits = pred[0]
    if start_logits.shape[0] == len(tokenized_datasets["validation"]):
        p, r = get_result(start_logits, end_logits, dataset["validation"], tokenized_datasets["validation"])
    else:
        p, r = get_result(start_logits, end_logits, dataset["test"], tokenized_datasets["test"])
    return evaluate_cmrc(p, r)

## step6 加载模型

In [43]:
model = AutoModelForQuestionAnswering.from_pretrained('hfl/chinese-macbert-base')

Some weights of BertForQuestionAnswering were not initialized from the model checkpoint at hfl/chinese-macbert-base and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


## step7 配置训练参数

In [48]:
args = TrainingArguments(
    output_dir="models_for_qa",
    per_device_train_batch_size=16,
    gradient_accumulation_steps=32,
    per_device_eval_batch_size=32,
    eval_strategy="steps",
    eval_steps=200,
    save_strategy="epoch",
    logging_steps=50,
    num_train_epochs=1,
    run_name='runs'
)

## step8 模型训练

In [49]:
trainer = Trainer(
    model=model,
    args=args,
    tokenizer=tokenizer,
    train_dataset=tokenized_datasets['train'],
    eval_dataset=tokenized_datasets['validation'],
    data_collator=DefaultDataCollator(),
    compute_metrics=metirc
)

C:\Users\fan\AppData\Local\Temp\ipykernel_30580\3959541405.py:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [50]:
trainer.train()

Step,Training Loss,Validation Loss


TrainOutput(global_step=38, training_loss=2.588596544767681, metrics={'train_runtime': 591.2409, 'train_samples_per_second': 32.455, 'train_steps_per_second': 0.064, 'total_flos': 3760517598755328.0, 'train_loss': 2.588596544767681, 'epoch': 1.0})

## step9 模型预测

In [51]:
from transformers import pipeline

pipe = pipeline("question-answering", model=model, tokenizer=tokenizer, device=0)
pipe

Device set to use cuda:0


In [55]:
pipe(question="《哈利波特》好看吗", context="《哈利波特》非常好看")

{'score': 0.1530991792678833, 'start': 0, 'end': 10, 'answer': '《哈利波特》非常好看'}